<a href="https://colab.research.google.com/github/T2718/AI_Make_Image/blob/main/AI_GUI_OverCount_LogIn/v3-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Driveのマウント
#@markdown **Driveを使用する場合はチェックを入れて実行**
drive_use = True #@param {type:"boolean"}
mount_point = '/content/drive'

#drive_use = Drive_path.strip() != ""
if drive_use:
  from google.colab import drive
  import os
  if not os.path.exists(mount_point):
    drive.mount(mount_point)
  if not os.path.exists(mount_point):
    drive_use = False
    print("Google Driveに接続できません。")

Mounted at /content/drive


In [2]:
#@markdown model_path_listを定義
from pathlib import Path

list_path = "/content/drive/MyDrive/Colab Notebooks/share_dir/AI_IMG/modelPathList.txt" #@param {type:"string"}
list_path = Path(list_path)

model_path_list = {}
explain_map = {}

with list_path.open(encoding="utf-8") as f:
    current_name = None
    for line in f:
        line = line.rstrip()  # 行末の空白だけ削除
        if not line:
            continue
        if line.startswith(" - [LogIn]"):  # 行頭の空白+ハイフン+空白+[LogIn]のみ判定
            if current_name:
                explain_map[current_name] = line
            continue
        if ":" in line:
            name, url = line.split(":", 1)
            current_name = name.strip()
            model_path_list[current_name] = url.strip()

# 見やすく出力（- [LogIn] のみ赤字）
print("model_path_list = {")
for k, v in model_path_list.items():
    explain = explain_map.get(k, "")
    if explain.startswith(" - [LogIn]"):  # 条件に合うものだけ赤字
        k_disp = f"\033[31m{k}\033[0m"
    else:
        k_disp = k
    print(f'    "{k_disp}": "{v}",')
print("}")


model_path_list = {
    "Ageless-v2": "https://civitai.com/api/download/models/74004?type=Model&format=SafeTensor&size=full&fp=fp16",
    "Animagine_XL-4.0": "https://civitai.com/api/download/models/1408658?type=Model&format=SafeTensor&size=full&fp=fp16",
    "Hassaku_XL-v3.0_WIP": "https://civitai.com/api/download/models/2010753?type=Model&format=SafeTensor&size=pruned&fp=fp16",
    "milktea-v3_typeB": "https://civitai.com/api/download/models/592037?type=Model&format=SafeTensor&size=pruned&fp=fp16",
}


In [14]:
#@title 変数の設定
#@markdown Google Driveに生成した画像を保存したい場合は<br>
#@markdown そのディレクトリのパスを入力。
#@markdown <br>Driveを使用しない際は空欄のまま<br>
#@markdown (例:/content/drive/MyDrive/Colab Notebooks/Generated_IMG)
Drive_path = "/content/drive/MyDrive/Colab Notebooks/share_dir/AI_IMG/generated" #@param {type:"string"}
#drive_use = Drive_path.strip() != ""
if drive_use:
  drive_save = Drive_path
else:
  drive_save = ""
#@markdown ##種類
#@markdown 1. **Hugging Face**
#@markdown 2. **CivitAI**<br>
#@markdown    [1,2について]Text to ImageまたはImage to Imageを<br>
#@markdown    行うためのモデルの種類<br>
#@markdown    **.safetensorsファイルを使用する際はHugging FaceでもCivitAIを選択！**<br>
#@markdown    例えば、Hugging Faceの<br>
#@markdown    eimiss/EimisAnimeDiffusion_2.0v<br>*https://huggingface.co/eimiss/EimisAnimeDiffusion_2.0v/resolve/main/EimisModel_2-0.safetensors?download=true*
#@markdown 3. **Resize&補完**#@markdown    元画像Pathに入れた、対応する画像をリサイズし、
#@markdown.   余白の部分を補完する。
#@markdown 4. **トークン数確認**
#@markdown   文のトークン数を確認できる。
#@markdown 5. **ダウンロード**
#@markdown 6. **削除**<br>
#@markdown   [4,5について]<small>*※Google Driveを使わずに画像生成した場合のみ有効*</small><br>
#@markdown   /content/downloadディレクトリを.zipでダウンロード<br>
#@markdown   または削除
#@markdown 7. **初期値**<br>
#@markdown   フォームの値の初期値を表示<br>
#@markdown   <small>ただし、フォームの値は置き換わりません。</small>
Kind = "CivitAI" #@param ["Hugging Face","CivitAI","Resize&補完","Token数確認","ダウンロード","削除","初期値"]
#@markdown ***以下、括弧()内に種類の番号ごとの入力必須の欄を指定***
#@markdown ##削除orダウンロードするパス (5,6)<br>
#@markdown 削除またはダウンロードするディレクトリのパス
DirPath = "/content/drive/MyDrive/Colab Notebooks/share_dir/AI_IMG/generated/GoodIMG/無題107_20250527013049.png" #@param {type:"string"}
#@markdown ##Text to Image(t2i)かImage to Image(i2i) (1,2)
SubMode = "t2i" #@param ["t2i","i2i"]
#@markdown ##ポジティブプロンプト (1,2,3,4)
Positive_Prompt = "masterpiece, illustration, show bare pussy, ahegao , sharp line, sharp focus,tentacles rape girl, tentacles in her mouth and pussy,  cat ears, kimono, tentacles grab arms and legs, orgasm, ecstasy, ertrag face, blush, massive Sticky white fluid whole body,massive liquid is sprout from pussy, spread legs" #@param {type:"string"}
#@markdown ##ネガティブプロンプト (1,2,3)
Negative_Prompt = "real, mono color, the tentacles are connected to the body,No tentacles in the mouth, mutated hands, ugly, bad anatomy, lowres, text, extra digit, missing limb" #@param {type:"string"}
#@markdown ##枚数 (1,2,3)
Num = 15 #@param {type:"number"}
#@markdown ##Step (1,2,3)
Steps = 60 #@param {type:"number"}
#@markdown ##Width,Height (1,2,3)
#@markdown - <small>*[1,2]のi2iの場合において、元画像のサイズで*</small><br>
#@markdown   <small>*生成する場合はチェックを入れてください。*</small>
OriginSize = True #@param {type:"boolean"}
#@markdown **どちらも8の倍数**で
Width = 768 #@param {type:"number"}
Height = 512 #@param {type:"number"}
#@markdown ##画像のPath ([1,2]のi2i,3)
ImagePath = "/content/drive/MyDrive/Colab Notebooks/share_dir/AI_IMG/generated/GoodIMG/Used_card/2 (1)-86x54.jpg" #@param {type:"string"}
#@markdown ##画像からの変化の強度(0~1の間) ([1,2]のi2i,3)
Strength = 0.8 #@param {type:"number"}
#@markdown ##MaskのPath (3)
MaskPath = "/content/mask 5.png" #@param {type:"string"}
#@markdown ##プロンプトに従う強度 (1,2)
Guidance = 9 #@param {type:"number"}
#@markdown ##モデルの状況 (2,3)
#@markdown - **none** インストールもしていない。
#@markdown - **origin** インストールはしているが<br>
#@markdown   変換はしていない。
#@markdown - **complete** 変換も済んでいる。そのまま使える。
Model_Status = "complete" #@param ["none","origin","complete"]
#@markdown ##Modelの種類<br>
#@markdown *(**1(=Hugging Face)**,2・3で先程の解答がnoneまたはoriginの時)*
#@markdown - 1(=Hugging Face)
#@markdown   - モデルの種類<br>
#@markdown     <small>以下、例</small>
#@markdown     - gsdf/Counterfeit-V3.0
#@markdown     - stablediffusionapi/eleet-model
#@markdown     - stablediffusionapi/brav6
#@markdown     - Vsukiyaki/ShiratakiMix
#@markdown     - stabilityai/stable-diffusion-xl-base-1.0
#@markdown - 2(=CivitAI)・3で、先程のモデルの状況で<br>
#@markdown   noneまたはoriginを選択した場合。<br>
#@markdown   <small>以下のどれかに従って入力。</small>
#@markdown   - 特定の数字を入力
#@markdown     1. Anime Pastel Dream<br>
#@markdown     <small>(SDXL:False,NeedYaml:False)</small>
#@markdown     2. WAI-NSFW-illustrious-SDXL v:11.0<br>
#@markdown     <small>(SDXL:True,NeedYaml:True)</small>
#@markdown   - \#◯◯◯◯(数字)<br>
#@markdown     Model ID
#@markdown   - CivitAIモデルのURL(https:~~)
#@markdown   更に、この2つ後の欄(.yaml)も選択必須
Model = "List:Animagine_XL-4.0" #@param {type:"string"}
#@markdown ##SDXLかどうか (2,3)
#@markdown ***'NoneType' object has no attribute 'tokenize'*<br>
#@markdown エラーが出た場合は、SDXLでない可能性があります。**
Sdxl = False #@param {type:"boolean"}
#@markdown ##.yamlファイルが必要かどうか([2,3]のSDXL)
#@markdown 三つ前の解答がnoneまたはoriginで、<br>
#@markdown SDXLである場合、選択必須<br>
#@markdown <small>Modelの.safetensorsに.yamlファイルが<br>
#@markdown 使用されていない場合必要。<br>
#@markdown 迷ったら入れてみよう。</small>
NeedYaml = True #@param {type:"boolean"}
#@markdown ##Pipelineをインストールするか(1,2,3)<br>
#@markdown **3の時はgeneratorをインストールするか**<br>
#@markdown ###以下の時はインストールする必要があります。<br>
#@markdown ###基本的に、続けて画像生成を行う際には必要ありません。
#@markdown - モデルの更新を行った時(インストールor変換)
#@markdown - 現セッションで初の生成の時
#@markdown - t2iまたはi2iでPipelineをインストール後、<br>
#@markdown   もう一方に切り替えて生成を行う時
PipeInstall = False #@param {type:"boolean"}
#@markdown ##ライブラリのインストールをするか<br>
#@markdown 現セッション内で一度でも[1,2,3]のどれかを<br>
#@markdown 実行した場合はライブラリのインストールが必要ありません。
NeedInstall = False #@param {type:"boolean"}
#@markdown ##Loraのパス(1,2)
#@markdown 使用する際はmodelのURLを入力。
#@markdown 使用しない場合は空欄のまま
LoraPath = "" #@param {type:"string"}
#@markdown ##別のモデルパスを指定 (origin, complete)
#@markdown originまたはcompleteで、別のパスからモデルを読み込みたい場合に指定。
#@markdown 空欄の場合はデフォルトのパスを使用。
Model_Insert_Path = "/content/drive/MyDrive/Colab Notebooks/share_dir/AI_IMG/model/milktea/v3_typeB/downloaded_model.safetensors" #@param {type:"string"}


#変数の変換 ------------------------------


# --- Drive設定 ---
#drive_use = Drive_path.strip() != ""

# --- モードを番号に変換 ---
mode = (
  "1" if Kind == "Hugging Face"
  else "2" if Kind == "CivitAI"
  else "3" if Kind == "Resize&補完"
  else "4" if Kind == "Token数確認"
  else "5" if Kind == "ダウンロード"
  else "6" if Kind == "削除"
  else "7" if Kind == "初期値"
  else ""
)

# --- パラメータ辞書の初期化 ---
hf_params = {}
civitai_params = {}

# --- Hugging Face ---
if mode == "1":
  hf_params = {
    "need_pipe": PipeInstall,
    "model_id": Model,
    "submode": "2" if SubMode == "i2i" else "1",
    "positive_prompt": Positive_Prompt,
    "negative_prompt": Negative_Prompt,
    "num": Num,
    "steps": Steps,
    "guidance": Guidance
  }
  if SubMode == "i2i":
    hf_params["strength"] = Strength
    hf_params["image_path"] = ImagePath

# --- CivitAI ---
elif mode == "2":
  Model_After = Model
  if Model.startswith("List:"):
    Model_After = model_path_list[Model[5:]]

  civitai_params = {
    "status": (
      "1" if Model_Status == "none"
      else "2" if Model_Status == "origin"
      else "3"
    ),
    "sdxl": Sdxl,
    "need_yaml": NeedYaml,
    #"need_yaml": (Model_Status in ["none", "origin"] and Sdxl),
    "need_pipe": PipeInstall,
    "submode": "2" if SubMode == "i2i" else "1",
    "positive_prompt": Positive_Prompt,
    "negative_prompt": Negative_Prompt,
    "num": Num,
    "steps": Steps,
    "guidance": Guidance,
    "url": Model_After,
    "lora_path": LoraPath.strip() if LoraPath.strip() else None,
    "model_insert_path": Model_Insert_Path.strip() if Model_Insert_Path.strip() else None
  }
  if SubMode == "i2i":
    civitai_params["strength"] = Strength
    civitai_params["image_path"] = ImagePath

elif mode == "3":
  resize_params = {
    "status": (
      "1" if Model_Status == "none"
      else "2" if Model_Status == "origin"
      else "3"
    ),
    "sdxl": Sdxl,
    "need_yaml": NeedYaml,
    #"need_yaml": (Model_Status in ["none", "origin"] and Sdxl),
    "need_pipe": PipeInstall,
    "submode": "2" if SubMode == "i2i" else "1",
    "positive_prompt": Positive_Prompt,
    "negative_prompt": Negative_Prompt,
    "num": Num,
    "steps": Steps,
    "url": Model,
    "strength" : Strength,
    "image_path" : ImagePath,
    "mask_path" : MaskPath,
    "install_tf" : PipeInstall,
    "model_insert_path": Model_Insert_Path.strip() if Model_Insert_Path.strip() else None
  }

# --- Token数確認 ---
elif mode == "4":
  token_prompt = Positive_Prompt

In [ ]:
#@title 実行する
#@markdown ##初回はライブラリのインストールに時間がかかります。<br>
#@markdown **特に、Kindが「1」または「2」での初回の実行では、<br>
#@markdown 以下の場合に時間を要する可能性があります。**
#@markdown - ModelStatus: 「none」または「origin」
#@markdown - NeedYaml: True
#@markdown - PipeInstall: True<br>

#@markdown *加えて、初回以降でもKindが「1」または「2」では、<br>
#@markdown 画像の生成に時間がかかる可能性があります。<br>
#@markdown 異常に実行時間が長い場合は以下の事項を確認してください。*
#@markdown - Model等のパスが正しいか
#@markdown - Modelや画像の容量が大きすぎないか(Model:最大7~8GBあたり)
#@markdown - Stepsが多すぎないか(Stepは通常50~80程度)
#@markdown - Errorが出ていないか
#@markdown - 入力すべき欄で書き忘れがないか
#@markdown - GPUを使用しているか
#@markdown - 回線が低速でないか
#@markdown - 容量(VRAM等)が不足していないか
#@markdown - Width,Heightが大きすぎないか(1024*1024ぐらいからかなり時間がかかる)

#@markdown また、SDXLを使用すると長くなる傾向にあります。
#@markdown ##生成が予想通りにいかない場合<br>
#@markdown 以下の点を確認してください。<br>
#@markdown - 正しくポジティブプロンプト、ネガティブプロンプトが<br>
#@markdown   記述できているか
#@markdown - Guidance,Stepsが小さすぎないか
#@markdown - **i2iの場合**
#@markdown   - 元画像のPathが正しいか
#@markdown   - Strengthが極端すぎないか<br>
#@markdown     <small>0で変化無し、1で元画像と全く異なる画像が生成されます。</small><br>
#@markdown   - Stepsが小さすぎないか
#@markdown - Pipelineが正しいか<br>
#@markdown   **<small>特に、t2iでインストールしたPipelineを<br>
#@markdown   i2iで使用するとt2iとして生成されます。<br>
#@markdown   出力の*総Steps数*が*Steps×Strength*と<br>
#@markdown   異なる場合はPipelineを<br>
#@markdown   再度インストールしてみてください。</small>**
model_list_H = ['stablediffusionapi/eleet-model',
              'stablediffusionapi/brav6',
              'Vsukiyaki/ShiratakiMix',
              'stabilityai/stable-diffusion-xl-base-1.0'
]

model_list_C = [["https://civitai.com/api/download/models/28100?type=Model&format=SafeTensor&size=full&fp=fp16","Anime Pastel Dream"],
                ["https://civitai.com/api/download/models/1410435?type=Model&format=SafeTensor&size=pruned&fp=fp16","WAI-NSFW-illustrious-SDXL v:11.0"]
]

mult = 3

#drive
"""
print("Google Driveを使用すると")
print("画像が一枚生成されるごとに保存されます。")
drive_use = input("Google Driveを使用しますか？ (y/n): ").strip().lower() == "y"
if drive_use:
  from google.colab import drive
  import os

  # マウントポイント
  mount_point = '/content/drive'

  # Driveがマウントされているか確認
  if not os.path.exists(mount_point):
    print("Google Driveに接続されていません。接続しますか？ (y/n)")
    connect_drive = input().strip().lower()
    if connect_drive == 'y':
      drive.mount(mount_point)
      print("Google Driveに接続しました。")
    else:
      print("Google Driveへの接続をスキップしました。")
      drive_use = False

if drive_use:
  print("保存する「ディレクトリ」のパスを入力してください。")
  print("(例: /content/drive/MyDrive/Colab Notebooks/your_dir)")
  print("※指定したディレクトリに追加で階層が生成されます(timestamp)。")
  print("保存しない場合は空欄で")
  drive_save = input("Path or Empty: ")
  if drive_save == "":
    drive_use = False

# ---モード選択---
print("=== モード選択 ===")
print("1: Hugging Faceで画像生成")
print("2: CivitAI（モデルDL / safetensors変換）")
print("3: トークン数を数える")
print("4: 出力フォルダをダウンロード(.zip)")
print("5: 出力フォルダを削除")
mode = input("番号を入力してください: ").strip()

# --- 必要なパラメータを先に全て入力 ---
hf_params = {}
civitai_params = {}





if mode == "1":
  hf_params["need_pipe"] = input("pipelineをインストールしますか？ (y/n): ").strip().lower() == "y"
  if hf_params["need_pipe"]:
    hf_params["model_id"] = input("HuggingFaceのモデルIDを入力（例: runwayml/stable-diffusion-v1-5）: ").strip()
  hf_params["submode"] = input("生成モードを選択（1: text2img, 2: img2img）: ").strip()
  hf_params["positive_prompt"] = input("プロンプトを入力: ").strip()
  hf_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ").strip()
  hf_params["num"] = int(input("生成する画像の枚数 : "))
  hf_params["steps"] = int(input("ステップ数（例: 70）: "))
  hf_params["guidance"] = float(input("guidance scale（例: 7.5）: "))
  if hf_params["submode"] == "2":
    hf_params["strength"] = float(input("strength (例: 0.75) : "))
    hf_params["image_path"] = input("初期画像のパスを入力してください（img2img用）: ")
"""
"""
  while True:
    print("修正する入力がある場合は")
    print("num:10")
    print("のように入力してください。")
    print("ない場合は空欄")
    corr = input("入力: ")
    if corr == "":
      break
    corr = corr.split(":")
    if corr[0] in hf_params:
      hf_params[corr[0]] = corr[1]
    elif corr[0] == "drive_use" and not corr[1] == "y":
      drive_use = False
    elif corr[0] == "drive_use" or corr[0] == "drive_save":
      from google.colab import drive
      import os
      # マウントポイント
      mount_point = '/content/drive'
      drive_use = True
      # Driveがマウントされているか確認
      if not os.path.exists(mount_point):
        print("Google Driveに接続されていません。接続しますか？ (y/n)")
        connect_drive = input().strip().lower()
        if connect_drive == 'y':
          drive.mount(mount_point)
          print("Google Driveに接続しました。")
        else:
          print("Google Driveへの接続をスキップしました。")
          drive_use = False
      if corr[1] == "":
        drive_use = False
      if corr[0] == "drive_use":
        print("保存する「ディレクトリ」のパスを入力してください。")
        print("(例: /content/drive/MyDrive/Colab Notebooks/your_dir)")
        print("※指定したディレクトリに追加で階層が生成されます(timestamp)。")
        print("保存しない場合は空欄で")
        drive_save = input("Path or Empty: ")
        if drive_save == "":
          drive_use = False
        else:
          drive_use = True

      if corr[0] == "drive_save" and drive_use:
        drive_save = corr[1]
"""
"""
elif mode == "2":
  print("--- CivitAI モデルの状況は？ ---")
  print("1: まだ何もしていない（DLも変換もしていない）")
  print("2: safetensorsファイルのみアップロード済み（変換が必要）")
  print("3: 変換済みでそのまま使える")
  civitai_params["status"] = input("番号を入力してください: ").strip()
  #civitai_params["model_name"] = input("保存先モデル名（例: models）: ").strip()
  if civitai_params["status"] == "1":
    for k in range(len(model_list_C)):
      print(f"{k+1} : {model_list_C[k][1]}")
    civitai_params["url"] = input("CivitAIのモデル/LoRA ダウンロードURL: ").strip()
    civitai_params["sdxl"] = input("モデルはSDXLですか？ (y/n): ").strip().lower() == "y"
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
  elif civitai_params["status"] == "2":
    print("モデルは")
    print("「/content/models/downloaded_model.safetensors」")
    print("に保存してください。")
    civitai_params["sdxl"] = input("モデルはSDXLですか？ (y/n): ").strip().lower() == "y"
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
  else:
    civitai_params["sdxl"] = input("モデルはSDXLですか？ (y/n): ")
  civitai_params["need_pipe"] = input("pipelineをインストールしますか？ (y/n): ").strip().lower() == "y"
  civitai_params["submode"] = input("生成モードを選択 (1: text2img, 2: img2img) : ").strip()
  civitai_params["positive_prompt"] = input("プロンプトを入力してください: ")
  civitai_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ")
  civitai_params["num"] = int(input("生成する画像の枚数 : "))
  civitai_params["steps"] = int(input("ステップ数（例: 70）: "))
  if civitai_params["submode"] == "2":
    civitai_params["strength"] = float(input("strength (例: 0.75)(img2img用) : "))
    civitai_params["image_path"] = input("初期画像のパスを入力してください（例: /content/your_img.png）: ")
  civitai_params["lora_path"] = input("LoRAを使用する場合はpathを(使わない場合は無入力)")
  if civitai_params["lora_path"] == "":
    civitai_params["lora_path"] = None


elif mode == "3":
  token_prompt = input("プロンプトを入力してください（トークン数確認）: ").strip()
"""

if mode == "4":
  print("少々お待ちください。")

elif mode == "5":
  confirm_delete = input("本当にoutputフォルダを削除しますか？ (y/n): ").strip().lower() == "y"



# --- 必要なinstallはここで実行（全てのinput終了後） ---
if mode in ["1","2","3","4"] and NeedInstall:
  print("インストールを開始します...")
  !pip install diffusers transformers accelerate safetensors

# Load Hugging Face token from secrets
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
except:
    HF_TOKEN = None
    print("Warning: Could not load Hugging Face token from Colab secrets.")


def tokenCounter(token_k):
  from transformers import CLIPTokenizer
  tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", token=HF_TOKEN)
  tokens = tokenizer(token_k)["input_ids"]
  return len(tokens)

# --- 各モード実行 ---
if mode == "1":
  pos_num = tokenCounter(hf_params["positive_prompt"])
  neg_num = tokenCounter(hf_params["negative_prompt"])
  print("Token数")
  print(" - Positive : "+str(pos_num))
  print(" - Negative : "+str(neg_num))
  mult = 1+max(pos_num, neg_num) // 77
  print(" - multiple : "+str(mult))

  from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
  from PIL import Image, ImageOps
  import torch
  import os
  from datetime import datetime, timedelta
  jst = datetime.utcnow() + timedelta(hours=9)
  timeDir = {
      "y":jst.strftime("%Y"),
      "m":jst.strftime("%m"),
      "d":jst.strftime("%d"),
      "t":jst.strftime("%H-%M-%S")
      }
  timestamp = jst.strftime("%Y%m%d_%H%M%S")
  torch_dtype = torch.float16

  if 'model_id' not in hf_params:
    hf_params["model_id"] = "unknown"
  if hf_params["need_pipe"]:
    if hf_params["submode"] == "1":
      pipe = StableDiffusionPipeline.from_pretrained(
        hf_params["model_id"],
        torch_dtype=torch.float16,
        safety_checker=None,  # Safe checkを無効に
        custom_pipeline="lpw_stable_diffusion"
      )
      pipe = pipe.to("cuda")  # GPUが必要です
    else:
      pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        hf_params["model_id"],
        torch_dtype=torch.float16,
        safety_checker=None,
        custom_pipeline="lpw_stable_diffusion"
      )
      pipe = pipe.to("cuda")
  if hf_params["submode"] != "1":
    init_image = Image.open(hf_params["image_path"]).convert("RGB")
    if not OriginSize:
      init_image = ImageOps.fit(init_image, (Width, Height), method=Image.Resampling.LANCZOS)
  for k in range(hf_params["num"]):
    if hf_params["submode"] == "1":

      image = pipe(hf_params["positive_prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   width=Width,
                   height=Height,
                   guidance_scale=hf_params["guidance"],
                   added_cond_kwargs={},
                   max_embeddings_multiples=mult
                ).images[0]
    else:
      #init_image = Image.open(hf_params["image_path"]).convert("RGB").resize((512, 512))
      image = pipe(prompt=hf_params["positive_prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   image=init_image,
                   strength=hf_params["strength"],
                   width=Width,
                   height=Height,
                   guidance_scale=hf_params["guidance"],
                   max_embeddings_multiples=mult
                ).images[0]
    if drive_use:
      save_dir = os.path.join(
          drive_save,
          timeDir["y"]+"-"+timeDir["m"],
          timeDir["d"],
          timeDir["t"]
      )

      os.makedirs(save_dir, exist_ok=True)
      image.save(save_dir+"/"+str(k+1)+".png")

      # プロンプトとネガティブプロンプトを保存（1回目のみ）
      if k == 0:
        if hf_params["submode"] != "1":
          init_image.save(save_dir+"/origin.png")
        with open(os.path.join(save_dir, "info.txt"), "w", encoding="utf-8") as f:
          if hf_params["submode"] == "1":
            f.write("t2i\n\n")
            f.write("Width,Height:\n"+str(Width)+','+str(Height)+'\n\n')
          else:
            f.write("i2i\n\n")
          f.write("Model:\nHugging Face / "+hf_params["model_id"]+"\n\n")
          f.write("Num_Inference_Steps:\n"+str(hf_params["steps"])+"\n\n")
          if hf_params["submode"] != "1":
            f.write("Strength:\n"+str(hf_params["strength"])+"\n\n")
          f.write("Positive Prompt:"+str(pos_num)+"\n" + hf_params["positive_prompt"] + "\n\n")
          f.write("Negative Prompt:"+str(neg_num)+"\n" + hf_params["negative_prompt"])

    else:
      os.makedirs("output/"+timestamp+"/"+hf_params["positive_prompt"][0:10], exist_ok=True)
      filename = f"output/"+timestamp+"/"+hf_params["positive_prompt"][0:10]+"/"+str(k+1)+".png"
      image.save(filename)
  print("画像を保存しました")

elif mode == "2":
  pos_num = tokenCounter(civitai_params["positive_prompt"])
  neg_num = tokenCounter(civitai_params["negative_prompt"])
  print("Token数")
  print(" - Positive : "+str(pos_num))
  print(" - Negative : "+str(neg_num))
  mult = 1+max(pos_num, neg_num) // 77
  print(" - multiple : "+str(mult))
  import os
  import torch
  import re
  from datetime import datetime, timedelta
  jst = datetime.utcnow() + timedelta(hours=9)
  timeDir = {
      "y":jst.strftime("%Y"),
      "m":jst.strftime("%m"),
      "d":jst.strftime("%d"),
      "t":jst.strftime("%H-%M-%S")
      }
  timestamp = jst.strftime("%Y%m%d_%H%M%S")
  if civitai_params["submode"] == "2":
    from PIL import Image, ImageOps
    from IPython.display import display


  base_dir = "/content/models"
  os.makedirs(base_dir, exist_ok=True)
  download_path = "/content/models/downloaded_model.safetensors"

  #※Driveのinfo.txtに書くための変数!!
  # modelName = "unknown"
  # if "Model" in locals():
  #   regex = r'([0-9]+)'
  #   regex2 = r'#(\d+)'
  #   if bool(re.match(regex, Model)):
  #     modelName = model_list_C[int(Model)-1][1]
  #   else:
  #     match = re.fullmatch(regex2, Model)
  #     if match:
  #       modelName = "model_id:" + match.group(1)
  #     else:
  #       modelName = Model


  if civitai_params["status"] == "1":
    regex = r'([0-9]+)'
    regex2 = r'#(\d+)'
    if bool(re.match(regex, civitai_params['url'])):
      civitai_params['url'] = model_list_C[int(civitai_params['url'])-1][0]
    else:
      match = re.fullmatch(regex2, civitai_params['url'])
      if match:
        civitai_params['url'] = 'https://civitai.com/api/download/models/'+match.group(1)

    model_url = civitai_params["url"]

    print(model_url)

    print(f"Downloading Checkpoint model to {download_path}...")
    !wget -O {download_path} "{model_url}"
    print(f"Download of safetensorFile complete!")


  lora_path = civitai_params["lora_path"] # LoRAはダウンロードしないとNone


  if civitai_params["status"] == "1" or civitai_params["status"] == "2":
    # Determine the checkpoint path for conversion
    conversion_checkpoint_path = civitai_params.get("model_insert_path")
    if not conversion_checkpoint_path or not os.path.exists(conversion_checkpoint_path):
        # If no insert path or path doesn't exist, use the downloaded path
        conversion_checkpoint_path = download_path

    if civitai_params["need_yaml"]:
      !wget -O /content/models/v1-inference.yaml \
https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
      #!wget -q https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-inference.yaml
    if not civitai_params["sdxl"]:
      !mkdir /content/converted
      !wget -O convert_original_stable_diffusion_to_diffusers.py \
      https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
      os.makedirs(base_dir, exist_ok=True)
      !python convert_original_stable_diffusion_to_diffusers.py \
        --checkpoint_path "{conversion_checkpoint_path}" \
        --original_config_file ./models/v1-inference.yaml \
        --dump_path ./converted/diffusers_model \
        --from_safetensors

  # Determine the model path to load for pipeline
  model_load_path = civitai_params.get("model_insert_path")
  if not model_load_path or not os.path.exists(model_load_path): # Add check for existence
      if civitai_params["status"] == "1" or civitai_params["status"] == "2":
          # If status is 1 or 2, and no insert path or path doesn't exist, use the downloaded/converted path
          model_load_path = "/content/converted/diffusers_model" if not civitai_params["sdxl"] else download_path
      elif civitai_params["status"] == "3":
          # If status is 3, use the default path for complete models
          model_load_path = "/content/converted/diffusers_model" if not civitai_params["sdxl"] else download_path # Assuming default path for complete models is the same

  #pipe = None
  if civitai_params["need_pipe"]:
    if civitai_params["sdxl"]:
      if os.path.isfile(model_load_path): # Check if it's a file
          from diffusers import StableDiffusionXLPipeline
          pipe_civitai = StableDiffusionXLPipeline.from_single_file(
              model_load_path, # Use model_load_path
              torch_dtype=torch.float16, # fp16モデルなので必須
              use_safetensors=True,      # safetensors形式であることを明示
              safety_checker=None,
              custom_pipeline="lpw_stable_diffusion"
          ).to("cuda")
      else: # Assume it's a directory
           from diffusers import StableDiffusionXLPipeline
           pipe_civitai = StableDiffusionXLPipeline.from_pretrained(
              model_load_path, # Use model_load_path
              torch_dtype=torch.float16, # fp16モデルなので必須
              safety_checker=None,
              custom_pipeline="lpw_stable_diffusion"
          ).to("cuda")
    else: # Not SDXL
      if os.path.isfile(model_load_path): # Check if it's a file
          from diffusers import StableDiffusionPipeline
          pipe_civitai = StableDiffusionPipeline.from_single_file(
              model_load_path, # Use model_load_path
              torch_dtype=torch.float16,
              use_safetensors=True, # Assuming single files are safetensors
              safety_checker=None,
              custom_pipeline="lpw_stable_diffusion"
          ).to("cuda")
      else: # Assume it's a directory
          from diffusers import StableDiffusionPipeline
          #pipe = StableDiffusionPipeline.from_pretrained(base_dir, torch_dtype=torch.float16).to("cuda")
          pipe_civitai = StableDiffusionPipeline.from_pretrained(
              model_load_path, # Use model_load_path
              torch_dtype=torch.float16,
              use_safetensors=False, # Assuming directories are not safetensors
              safety_checker=None,
              custom_pipeline="lpw_stable_diffusion"
            ).to("cuda")
    # パイプラインを作成した直後に以下を追加します
    if lora_path:
      pipe_civitai.load_lora_weights(lora_path, use_safetensors=True)

  modelName_civitai = model_load_path if model_load_path and os.path.exists(model_load_path) else (pipe_civitai.pretrained_model_name_or_path if hasattr(pipe_civitai, 'pretrained_model_name_or_path') else "unknown")


  if civitai_params["submode"] == "1":
  #pipe.enable_xformers_memory_efficient_attention()
    for k in range(civitai_params["num"]):
      image = pipe_civitai(prompt=civitai_params["positive_prompt"],
          negative_prompt=civitai_params["negative_prompt"],
          num_inference_steps=civitai_params["steps"],
          width=Width,
          height=Height,
          guidance_scale=civitai_params["guidance"],
          max_embeddings_multiples=mult
      ).images[0]
      if drive_use:
        save_dir = os.path.join(
            drive_save,
            timeDir["y"]+"-"+timeDir["m"],
            timeDir["d"],
            timeDir["t"]
          )
        os.makedirs(save_dir, exist_ok=True)
        image.save(save_dir+"/"+str(k+1)+".png")

        # プロンプトとネガティブプロンプトを保存（1回目のみ）
        if k == 0:
          with open(os.path.join(save_dir, "info.txt"), "w", encoding="utf-8") as f:
            f.write("t2i\n\n")
            f.write("Width,Height:\n"+str(Width)+","+str(Height)+"\n\n")
            if civitai_params["sdxl"]:
              f.write("Model:\nCivitAI(SDXL) / "+modelName_civitai+"\n\n")
            else:
              f.write("Model:\nCivitAI / "+modelName_civitai+"\n\n")
            f.write("Num_Inference_Steps:\n"+str(civitai_params["steps"])+"\n\n")
            f.write("Positive Prompt:"+str(pos_num)+"\n" + civitai_params["positive_prompt"] + "\n\n")
            f.write("Negative Prompt:"+str(pos_num)+"\n" + civitai_params["negative_prompt"])

      else:
        os.makedirs("output/t2i/"+timestamp+"/"+civitai_params["positive_prompt"][0:10], exist_ok=True)
        image.save("output/t2i/"+timestamp+"/"+civitai_params["positive_prompt"][0:10]+"/"+str(k+1)+".png")
    print("画像を保存しました: output/t2i/"+timestamp+"/"+civitai_params["positive_prompt"][0:10])
  else:
    init_image = Image.open(civitai_params["image_path"]).convert("RGB")
    img_width, img_height = init_image.size
    if not OriginSize:
      init_image = ImageOps.fit(init_image, (Width, Height), method=Image.Resampling.LANCZOS)
      img_width = Width
      img_height = Height
    #display(init_image)
    for k in range(civitai_params["num"]):
      image = pipe_civitai(prompt=civitai_params["positive_prompt"],
          negative_prompt=civitai_params["negative_prompt"],
          image=init_image,
          num_inference_steps=civitai_params["steps"],
          strength=civitai_params["strength"],
          width=img_width,
          height=img_height,
          guidance_scale=civitai_params["guidance"],
          max_embeddings_multiples=mult
      ).images[0]
      if drive_use:
        save_dir = os.path.join(
            drive_save,
            timeDir["y"]+"-"+timeDir["m"],
            timeDir["d"],
            timeDir["t"]
          )
        os.makedirs(save_dir, exist_ok=True)
        image.save(save_dir+"/"+str(k+1)+".png")

        # プロンプトとネガティブプロンプトを保存（1回目のみ）
        if k == 0:
          init_image.save(save_dir+"/origin.png")
          with open(os.path.join(save_dir, "info.txt"), "w", encoding="utf-8") as f:
            f.write("i2i\n\n")
            if civitai_params["sdxl"]:
              f.write("Model:\nCivitAI(SDXL) / "+modelName_civitai+"\n\n")
            else:
              f.write("Model:\nCivitAI / "+modelName_civitai+"\n\n")
            f.write("Num_Inference_Steps:\n"+str(civitai_params["steps"])+"\n\n")
            f.write("Strength:\n"+str(civitai_params["strength"])+"\n\n")
            f.write("Size:\nw..."+str(img_width)+",h..."+str(img_height)+"\n\n")
            f.write("Positive Prompt:"+str(pos_num)+"\n" + civitai_params["positive_prompt"] + "\n\n")
            f.write("Negative Prompt:"+str(neg_num)+"\n" + civitai_params["negative_prompt"])
      else:
        os.makedirs("output/i2i/"+timestamp+"/"+civitai_params["positive_prompt"][0:10], exist_ok=True)
        image.save("output/i2i/"+timestamp+"/"+civitai_params["positive_prompt"][0:10]+"/"+str(k+1)+".png")
    print("画像を保存しました: output/i2i/"+timestamp+"/"+civitai_params["positive_prompt"][0:10])

  """
  resize_params = {
    "status": (
      "1" if Model_Status == "none"
      else "2" if Model_Status == "origin"
      else "3"
    ),
    "sdxl": Sdxl,
    "need_yaml": NeedYaml,
    #"need_yaml": (Model_Status in ["none", "origin"] and Sdxl),
    "need_pipe": PipeInstall,
    "submode": "2" if SubMode == "i2i" else "1",
    "positive_prompt": Positive_Prompt,
    "negative_prompt": Negative_Prompt,
    "num": Num,
    "steps": Steps,
    "url": Model,
    "strength" = Strength,
    "image_path = ImagePath,
  }
  """
elif mode == "3":
  #マスク作成


  #既にインストールされてるから要らない
  #!pip install pillow

  from PIL import Image, ImageOps
  from diffusers import StableDiffusionControlNetInpaintPipeline, ControlNetModel, DDIMScheduler
  from diffusers.utils import load_image
  import numpy as np
  import torch
  import os
  from datetime import datetime, timedelta
  jst = datetime.utcnow() + timedelta(hours=9)
  timeDir = {
      "y":jst.strftime("%Y"),
      "m":jst.strftime("%m"),
      "d":jst.strftime("%d"),
      "t":jst.strftime("%H-%M-%S")
      }
  timestamp = jst.strftime("%Y%m%d_%H%M%S")

  pos_num = "?"
  neg_num = "?"

  modelName_resize_base = "runwayml/stable-diffusion-v1-5"

  # ----------- 変数定義 -----------
  #path = resize_params[
  w, h = Width, Height  # 出力サイズ
  # --------------------------------

  # 元画像読み込み
  img = Image.open(resize_params["image_path"]).convert("RGB")
  w0, h0 = img.size
  rw, rh = w / w0, h / h0

  # サイズに応じてリサイズ＆パディング
  if resize_params["mask_path"] == "":
    if rw < rh:
      new_w = w
      new_h = int(h0 * rw)
      resized = img.resize((new_w, new_h), resample=Image.Resampling.LANCZOS)
      pad_top = (h - new_h) // 2
      pad_bottom = h - new_h - pad_top
      pad = (0, pad_top, 0, pad_bottom)
    else:
      new_h = h
      new_w = int(w0 * rh)
      resized = img.resize((new_w, new_h), resample=Image.Resampling.LANCZOS)
      pad_left = (w - new_w) // 2
      pad_right = w - new_w - pad_left
      pad = (pad_left, 0, pad_right, 0)

    # 余白追加済み画像の生成
    init_image = ImageOps.expand(resized, border=pad, fill=(255, 255, 255))


    # マスク画像（余白のみ白、他は黒）
    mask_image = Image.new("L", (w, h), color=255)
    mask_image.paste(0, box=(pad[0], pad[1], w - pad[2], h - pad[3]))
  else:
    init_image = img
    mask_image = load_image(
      resize_params["mask_path"]
    )
    mask_image = Image.open(resize_params["mask_path"]).convert("L")
    mask_image = mask_image.resize((w0,h0))
  #mask_image.paste(0, box=(pad[0], pad[1], w - pad[2], h - pad[3]))
  # 保存

  #os.makedirs("/content/gen",exist_ok=True)
  #init_image.save("gen/padded_image.png")
  #mask_image.save("gen/mask_image.png")

  #install_tf = False


  #if install_tf:
  #  !pip install transformers accelerate



  """init_image = load_image(
    #"https://huggingface.co/datasets/diffusers/test-arrays/resolve/main/stable_diffusion_inpaint/boy.png"
    "/content/gen/padded_image.png"
  )"""
  #init_image = init_image.resize((512, 512))

  if resize_params["install_tf"]:
    generator = torch.Generator(device="cpu").manual_seed(1)


  #mask_image = mask_image.resize((512, 512))


  def make_inpaint_condition(image, image_mask):
    image = np.array(image.convert("RGB")).astype(np.float32) / 255.0
    image_mask = np.array(image_mask.convert("L")).astype(np.float32) / 255.0

    assert image.shape[0:1] == image_mask.shape[0:1], "image and image_mask must have the same image size"
    image[image_mask > 0.5] = -1.0  # set as masked pixel
    image = np.expand_dims(image, 0).transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return image


  control_image = make_inpaint_condition(init_image, mask_image)

  controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_inpaint", torch_dtype=torch.float16
  )

  # Determine the model path to load for resize
  model_load_path_resize = resize_params.get("model_insert_path")
  if not model_load_path_resize or not os.path.exists(model_load_path_resize): # Add check for existence
      # If no insert path or path doesn't exist, use the default base model
      model_load_path_resize = modelName_resize_base


  if resize_params["install_tf"]:
    if os.path.isfile(model_load_path_resize): # Check if it's a file
        pipe_resize = StableDiffusionControlNetInpaintPipeline.from_single_file(
            model_load_path_resize, controlnet=controlnet, torch_dtype=torch.float16, # Use model_load_path_resize
            safety_checker=None,
            use_safetensors=True, # Assuming single files are safetensors
        )
    else: # Assume it's a directory or huggingface model id
        pipe_resize = StableDiffusionControlNetInpaintPipeline.from_pretrained(
            model_load_path_resize, controlnet=controlnet, torch_dtype=torch.float16, # Use model_load_path_resize
            safety_checker=None,
        )


    pipe_resize.scheduler = DDIMScheduler.from_config(pipe_resize.scheduler.config)
    pipe_resize.enable_model_cpu_offload()

  modelName_resize = model_load_path_resize if model_load_path_resize and os.path.exists(model_load_path_resize) else (pipe_resize.pretrained_model_name_or_path if hasattr(pipe_resize, 'pretrained_model_name_or_path') else "unknown")

  #positive_prompt = "anime, colorful eyes, 1girl, floating, golden fox ears, golden long hair, elegant face, gentle smile, modest dress, natural fingers, starlit sky, magical glow, colorful light particles, soft rainbow haze, aurora sky, bluish shadows, faint magenta glow, delicate warm light"
  #negative_prompt = "bad hands, extra fingers, fused fingers, deformed hands, missing fingers, worst quality, low quality"
  #positive_prompt = resize_params["positive_prompt"]
  #negative_prompt = resize_params["negative_prompt"]
  #num = 10
  #steps = 100

  #os.makedirs("/content/result/"+prompt[0:10], exist_ok=True)

  # generate image
  for k in range(resize_params["num"]):
    image = pipe_resize(
      resize_params["positive_prompt"],
      negative_prompt=resize_params["negative_prompt"],
      num_inference_steps=resize_params["steps"],
      generator=generator,
      eta=1.0,
      image=init_image,
      mask_image=mask_image,
      control_image=control_image,

    ).images[0]

    #init_image = Image.open(civitai_params["image_path"]).convert("RGB")
    #init_image = ImageOps.fit(init_image, (1024, 1024), method=Image.Resampling.LANCZOS)
    #display(init_image)
    #for k in range(civitai_params["num"]):
    """image = pipe_civitai(prompt=civitai_params["positive_prompt"],
          negative_prompt=civitai_params["negative_prompt"],
          image=init_image,
          num_inference_steps=civitai_params["steps"],
          strength=civitai_params["strength"],
          width=Width,
          height=Height,
          guidance_scale=civitai_params["guidance"],
    ).images[0]"""
    if drive_use:
      save_dir = os.path.join(
          drive_save,
          timeDir["y"]+"-"+timeDir["m"],
          timeDir["d"],
          timeDir["t"]
        )
      os.makedirs(save_dir, exist_ok=True)
      image.save(save_dir+"/"+str(k+1)+".png")

      # プロンプトとネガティブプロンプトを保存（1回目のみ）
      if k == 0:
        img.save(save_dir+"/origin.png")
        mask_image.save(save_dir+"/mask.png")
        with open(os.path.join(save_dir, "info.txt"), "w", encoding="utf-8") as f:
          f.write("resize\n\n")
          #if civitai_params["sdxl"]:
          #  f.write("Model:\nCivitAI(SDXL) / "+modelName+"\n\n")
          #else:
          #  f.write("Model:\nCivitAI / "+modelName+"\n\n")
          if resize_params["mask_path"] == "":
            f.write("Width, Height:\nw..."+str(w)+", h..."+str(h)+"\n\n")
          else:
            f.write("Width, Height:\nw..."+str(w0)+", h..."+str(h0)+"\n\n")
          f.write("Model:\n"+modelName_resize+"\n\n")
          f.write("Num_Inference_Steps:\n"+str(resize_params["steps"])+"\n\n")
          #f.write("Strength:\n"+str(civitai_params["strength"])+"\n\n")
          f.write("Positive Prompt:\n" + resize_params["positive_prompt"] + "\n\n")
          f.write("Negative Prompt:\n" + resize_params["negative_prompt"])
    else:
      os.makedirs("output/i2i/"+timestamp+"/"+resize_params["positive_prompt"][0:10], exist_ok=True)
      image.save("output/i2i/"+timestamp+"/"+resize_params["positive_prompt"][0:10]+"/"+str(k+1)+".png")
  print("画像を保存しました: output/i2i/"+timestamp+"/"+resize_params["positive_prompt"][0:10])

  #pipe.safety_checker = lambda images, **kwargs: (images, [False] * len(images))
  #image.save("result/"+prompt[0:10]+"/"+str(k)+".png")

elif mode == "4":
  print("Token数 : "+str(tokenCounter(token_prompt)))

elif mode == "5":
  import shutil
  from google.colab import files

  # 圧縮したいディレクトリのパス
  folder_path = DirPath
  zip_filename = folder_path + '.zip'

  # ディレクトリを zip 形式で圧縮
  shutil.make_archive(folder_path, 'zip', folder_path)

  # Colab にダウンロードリンクを表示
  files.download(zip_filename)


elif mode == "6":
  import shutil
  if confirm_delete:
    if os.path.exists(DirPath) and os.path.isdir(DirPath):
      shutil.rmtree(DirPath)
      print(f"{DirPath} を削除しました。")
    else:
      print(f"{DirPath} は存在しないか、ディレクトリではありません。")
  else:
    print("キャンセルされました")

elif mode == "7":
  default_params = {
    "Drive_path": "",
    "Kind": "Hugging Face",
    "SubMode": "t2i",
    "Prompt": "",
    "Negative_Prompt": "",
    "Num": 10,
    "Steps": 50,
    "ImagePath": "",
    "Strength": 0.75,
    "Guidance": 7.5,
    "Model_Status": "none",
    "Model": "2",
    "Sdxl": True,
    "PipeInstall": True,
    "LoraPath": ""
  }

  for k in default_params.keys():
    if type(default_params[k]) == int or type(default_params[k]) == float:
      print(k+" : "+str(default_params[k]))
    elif type(default_params[k]) == bool:
      print(k+" : "+str(default_params[k]))
    else:
      print(k+' : "'+default_params[k]+'"')

Token数
 - Positive : 70
 - Negative : 42
 - multiple : 1


/tmp/ipython-input-1631049795.py:346: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  jst = datetime.utcnow() + timedelta(hours=9)


  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

...

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

In [50]:
#@title LogInが必要な場合に、modelをダウンロードするコード(実行後は"origin"からスタート)。
#@markdown CivitAIでAPIキーを生成(名前はなんでもいい)して、seacretにCIVITAI_APIという名前でvalueにtoken追加して。

# Colab 用：CivitAI ダウンロード（SafeTensor, fp16 など）
# インデントは 2 スペースに統一しています。
import requests
from pathlib import Path
from tqdm import tqdm
from google.colab import userdata

# --- 設定 (ここを編集) ---
TOKEN_NAME = "CIVITAI_API" #@param {type:"string"}
DOWNLOAD_URL = "https://civitai.com/api/download/models/74004?type=Model&format=SafeTensor&size=full&fp=fp16" #@param {type:"string"}
OUTPUT_DIR = "/content/models" #@param {type:"string"}
OUTPUT_FILE = "downloaded_model.safetensors" #@param {type:"string"}
# -------------------------

OUTPUT_DIR = Path(OUTPUT_DIR)
OUTPUT_FILE = OUTPUT_DIR / OUTPUT_FILE

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

headers = {
  "Authorization": f"Bearer {userdata.get(TOKEN_NAME)}",
  "User-Agent": "colab-civitai-downloader/1.0"
}

print("ダウンロードを開始します。URL:", DOWNLOAD_URL)

with requests.get(DOWNLOAD_URL, headers=headers, stream=True, allow_redirects=True, timeout=60) as r:
  status = r.status_code
  if status == 401 or status == 403:
    raise SystemExit(f"認証エラー: HTTP {status} — APIキーの権限または形式を確認してください。")
  if status == 404:
    raise SystemExit("ファイルが見つかりません: HTTP 404。モデルの存在やアクセス権を確認してください。")
  if status >= 500:
    raise SystemExit(f"サーバーエラー: HTTP {status}。時間をおいて再試行してください。")

  total = int(r.headers.get("Content-Length", 0))
  chunk_size = 8192

  print(f"保存先: {OUTPUT_FILE} (推定サイズ: {total} bytes)")

  with open(OUTPUT_FILE, "wb") as f, tqdm(total=total, unit="B", unit_scale=True, desc="downloaded_model.safetensors", ncols=80) as pbar:
    for chunk in r.iter_content(chunk_size=chunk_size):
      if chunk:
        f.write(chunk)
        pbar.update(len(chunk))

print("ダウンロード完了:", OUTPUT_FILE)


ダウンロードを開始します。URL: https://civitai.com/api/download/models/74004?type=Model&format=SafeTensor&size=full&fp=fp16
保存先: /content/models/downloaded_model.safetensors (推定サイズ: 4244124026 bytes)


downloaded_model.safetensors: 100%|████████| 4.24G/4.24G [01:10<00:00, 60.4MB/s]

ダウンロード完了: /content/models/downloaded_model.safetensors
